<a href="https://colab.research.google.com/github/Ruchiraongithub/Comment_toxicity_prediction/blob/main/Comment_Toxicity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
data=pd.read_csv("/content/Toxic_comments_data.zip")


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56745 entries, 0 to 56744
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  56745 non-null  int64 
 1   Toxicity    56745 non-null  int64 
 2   tweet       56745 non-null  object
dtypes: int64(2), object(1)
memory usage: 1.3+ MB


In [ ]:
data.head(5)

,Unnamed: 0,Toxicity,tweet
0,0,0,@user when a father is dysfunctional and is s...
1,1,0,@user @user thanks for #lyft credit i can't us...
2,2,0,bihday your majesty
3,3,0,#model i love u take with u all the time in ...
4,4,0,factsguide: society now #motivation


In [ ]:
data=data.drop("Unnamed: 0", axis=1)

In [ ]:
data.head(5)

,Toxicity,tweet
0,0,@user when a father is dysfunctional and is s...
1,0,@user @user thanks for #lyft credit i can't us...
2,0,bihday your majesty
3,0,#model i love u take with u all the time in ...
4,0,factsguide: society now #motivation


In [ ]:
data['Toxicity'].value_counts()

,count
Toxicity,
0,32592
1,24153


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('average_perceptron_tagger')
from nltk import WordNetLemmatizer
from nltk import pos_tag, word_tokenize
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Error loading average_perceptron_tagger: Package
[nltk_data]     'average_perceptron_tagger' not found in index


# **Text Preprocessing**

In [ ]:
wordnet_lemmatizer = WordNetLemmatizer()

In [ ]:
import re

In [ ]:
!pip install nltk
import nltk
nltk.download('averaged_perceptron_tagger')
import re
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag


wordnet_lemmatizer = WordNetLemmatizer()


def prepare_text(text):
    def get_wordnet_pos(treebank_tag):
        if treebank_tag.startswith('J'):
            return wordnet.ADJ
        elif treebank_tag.startswith('V'):
            return wordnet.VERB
        elif treebank_tag.startswith('N'):
            return wordnet.NOUN
        elif treebank_tag.startswith('R'):
            return wordnet.ADV
        else:
            return wordnet.NOUN

    text = re.sub(r'[^a-zA-Z\']', ' ', text)
    text = text.split()
    text = " ".join(text)
    text = text.lower()
    text = word_tokenize(text)
    text = pos_tag(text)
    lemma = []
    for i in text:
        lemma.append(wordnet_lemmatizer.lemmatize(i[0], pos=get_wordnet_pos(i[1])))
    lemma = " ".join(lemma)
    return lemma


data['clean_tweet'] = data['tweet'].apply(lambda x: prepare_text(x))

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
data.head(5)

,Toxicity,tweet,clean_tweet
0,0,@user when a father is dysfunctional and is s...,user when a father be dysfunctional and be so ...
1,0,@user @user thanks for #lyft credit i can't us...,user user thanks for lyft credit i ca n't use ...
2,0,bihday your majesty,bihday your majesty
3,0,#model i love u take with u all the time in ...,model i love u take with u all the time in ur
4,0,factsguide: society now #motivation,factsguide society now motivation


# **TF-IDF for features**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

In [ ]:
corpus=data['clean_tweet'].values.astype('U')

In [ ]:
stopwords=set(nltk_stopwords.words('english'))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
stopwords_list = list(stopwords)

count_tf_idf = TfidfVectorizer(stop_words=stopwords_list)

tf_idf = count_tf_idf.fit_transform(corpus)

In [ ]:
tf_idf_train, tf_idf_test, target_train, target_test = train_test_split(tf_idf, data['Toxicity'], test_size=0.8, random_state=42, shuffle=True)

In [ ]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Separate features and labels
texts = data['clean_tweet'].values  # Preprocessed text
labels = data['Toxicity'].values   # Binary labels: 0 (non-toxic) and 1 (toxic)

# Tokenization and padding parameters
max_words = 10000  # Vocabulary size
max_len = 100      # Maximum sequence length for padding

# Tokenize the text
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# Pad sequences to ensure uniform input shape
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post')

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout

# Model parameters
embedding_dim = 50  # Embedding dimension

# Build the CNN model
model = Sequential([
    Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len),
    Conv1D(filters=128, kernel_size=5, activation='relu'),
    GlobalMaxPooling1D(),
    Dropout(0.5),
    Dense(10, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Sigmoid for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])




/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
# Training parameters
batch_size = 32
epochs = 10

history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)


Epoch 1/10
1135/1135 ━━━━━━━━━━━━━━━━━━━━ 34s 28ms/step - accuracy: 0.8079 - loss: 0.3851 - val_accuracy: 0.9433 - val_loss: 0.1526
Epoch 2/10
1135/1135 ━━━━━━━━━━━━━━━━━━━━ 39s 26ms/step - accuracy: 0.9415 - loss: 0.1576 - val_accuracy: 0.9434 - val_loss: 0.1633
Epoch 3/10
1135/1135 ━━━━━━━━━━━━━━━━━━━━ 30s 27ms/step - accuracy: 0.9535 - loss: 0.1278 - val_accuracy: 0.9411 - val_loss: 0.1728
Epoch 4/10
1135/1135 ━━━━━━━━━━━━━━━━━━━━ 41s 26ms/step - accuracy: 0.9654 - loss: 0.0951 - val_accuracy: 0.9378 - val_loss: 0.2099
Epoch 5/10
1135/1135 ━━━━━━━━━━━━━━━━━━━━ 42s 28ms/step - accuracy: 0.9711 - loss: 0.0754 - val_accuracy: 0.9355 - val_loss: 0.2290
Epoch 6/10
1135/1135 ━━━━━━━━━━━━━━━━━━━━ 39s 26ms/step - accuracy: 0.9748 - loss: 0.0618 - val_accuracy: 0.9345 - val_loss: 0.3117
Epoch 7/10
1135/1135 ━━━━━━━━━━━━━━━━━━━━ 41s 26ms/step - accuracy: 0.9793 - loss: 0.0480 - val_accuracy: 0.9303 - val_loss: 0.3582
Epoch 8/10
1135/1135 ━━━━━━━━━━━━━━━━━━━━ 41s 26ms/step - accuracy: 0.9809 -

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")


355/355 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9315 - loss: 0.5061
Test Accuracy: 0.93


In [ ]:
# Function to predict if a sentence is toxic or non-toxic
def predict_toxicity(sentence, model, tokenizer, max_len=100):
    # Tokenize and pad the input sentence
    sequence = tokenizer.texts_to_sequences([sentence])
    padded_sequence = pad_sequences(sequence, maxlen=max_len, padding='post')

    # Make a prediction
    prediction = model.predict(padded_sequence)[0][0]

    # Interpret the prediction
    if prediction < 0.05:
        return "Non-Toxic"
    else:
        return "Toxic"

# Example of using the function with user input
user_sentence = input("Enter a sentence to check if it's toxic or non-toxic: ")
result = predict_toxicity(user_sentence, model, tokenizer)
print(f"The sentence is: {result}")


Enter a sentence to check if it's toxic or non-toxic: you are an idiot
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
The sentence is: Toxic
